simple, fast and lighweight example of refining a DistillBert LLM model with new data

In [1]:
#!pip install datasets
# !pip install transformers

In [2]:

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import pprint


In [3]:
# Load a small dataset for demonstration (replace with your data)

# GLUE (General Language Understanding Evaluation) benchmark
# the MRPC (Microsoft Research Paraphrase Corpus)

# only a few percent training data
dataset = load_dataset("glue", "mrpc", split="train[:2%]")

print(dataset,"\n")
# print(dataset[0])
pprint.pprint(dataset[0])
print()
pprint.pprint(dataset[1])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 73
}) 

{'idx': 0,
 'label': 1,
 'sentence1': 'Amrozi accused his brother , whom he called " the witness " , '
              'of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his '
              'brother of deliberately distorting his evidence .'}

{'idx': 1,
 'label': 0,
 'sentence1': "Yucaipa owned Dominick 's before selling the chain to Safeway "
              'in 1998 for $ 2.5 billion .',
 'sentence2': "Yucaipa bought Dominick 's in 1995 for $ 693 million and sold "
              'it to Safeway for $ 1.8 billion in 1998 .'}


In [4]:
# Initialize the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2) # Assuming binary classification

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

for idx in range(2):
    print()
    for k, v in tokenized_dataset[0].items():
      print(f'{k}: {v}')
print()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



sentence1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
sentence2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
label: 1
idx: 0
input_ids: [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:

# Define training arguments (adjust these as needed)
training_args = TrainingArguments(
    run_name="distilbert_svw1",
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10
)

# Create Trainer instance
# Using the same dataset for evaluation,
# just a simple demo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_distilbert")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: seanvw (seanvw-none). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10,0.751600,0.744801
